In [ ]:
'''
!pip -q install datasets
!pip -q install loralib
!pip -q install sentencepiece
!pip -q install bitsandbytes
!pip -q install accelerate

'''

In [ ]:
'''

!pip install git+https://github.com/huggingface/transformers
!pip -q install git+https://github.com/huggingface/peft.git

'''

In [ ]:
# from peft import PeftModel
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig

tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-7b-hf")
model = LlamaForCausalLM.from_pretrained("decapoda-research/llama-7b-hf")
# model = PeftModel.from_pretrained(model, "tloen/alpaca-lora-7b")

In [ ]:
def generate_prompt(text):
  inputs = tokenizer(text,return_tensors='pt')
  input_ids = inputs['input_ids'].cuda()

  generation_config = GenerationConfig(
      temperature=0.6,
      top_p=0.95,
      repetition_penalty=1.2
  )

  print('creating a response')

  generation_output = model.generate(
      input_ids = input_ids,
      generation_config = generation_config,
      return_dict_in_generate = True,
      output_scores=True,
      max_new_tokens=256
  )

  for s in generation_output.sequences:
    output = tokenizer.decode(s)
    print('response: ', output.split('### Response:')[1].strip() )


In [ ]:
input_text = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
could you write some python code to print the sequence of fibonacci ?

### Response:"""
generate_prompt(input_text)